In [1]:
from pandas.io.json import json_normalize
import pandas as pd
from pandasql import sqldf
# SQLITE

import warnings
warnings.filterwarnings("ignore")

# import data + table/data cleaning

## brands

In [15]:
# import brands 
brands = open("brands.json", "r")

df_brands = pd.DataFrame()

while True:
    try:           # read line by line
        line = brands.readline()
        data = json.loads(line)
    
        temp = pd.json_normalize(data)
        df_brands = df_brands.append(temp)
    
    except:        # break if last line
        break

In [16]:
# rename columns
df_brands.rename(columns={'_id.$oid': '_id'}, inplace=True)

# reorder columns
column_names = ['_id', 'barcode', 'brandCode',
                'category', 'categoryCode', 'cpg.$id.$oid',
                'cpg.$ref', 'topBrand', 'name']

brands = df_brands.reindex(columns=column_names)

brands

,_id,barcode,brandCode,category,categoryCode,cpg.$id.$oid,cpg.$ref,topBrand,name
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,False,test brand @1612366101024
0,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,False,Starbucks
0,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146176
0,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,False,test brand @1612366146051
0,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,False,test brand @1612366146827
...,...,...,...,...,...,...,...,...,...
0,5f77274dbe37ce6b592e90c0,511111116752,NaN,Baking,BAKING,5f77274dbe37ce6b592e90bf,Cogs,NaN,test brand @1601644365844
0,5dc1fca91dda2c0ad7da64ae,511111706328,DIPPIN DOTS CEREAL,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Cogs,NaN,Dippin DotsÂ® Cereal
0,5f494c6e04db711dd8fe87e7,511111416173,TEST BRANDCODE @1598639215217,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,NaN,test brand @1598639215217
0,5a021611e4b00efe02b02a57,511111400608,LIPTON TEA Leaves,Grocery,NaN,5332f5f6e4b03c9a25efd0b4,Cogs,False,LIPTON TEA Leaves


In [81]:
brands.to_csv('brands.csv', index=False)

In [36]:
# null check
brands.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1167 entries, 0 to 0
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           1167 non-null   object
 1   barcode       1167 non-null   object
 2   brandCode     933 non-null    object
 3   category      1012 non-null   object
 4   categoryCode  517 non-null    object
 5   cpg.$id.$oid  1167 non-null   object
 6   cpg.$ref      1167 non-null   object
 7   topBrand      555 non-null    object
 8   name          1167 non-null   object
dtypes: object(9)
memory usage: 91.2+ KB


In [70]:
# duplicate check
brands.duplicated().sum()

0

## receipts

In [74]:
# import receipts
receipts = open("receipts.json", "r")

df_receipts = pd.DataFrame()

while True:
    try:           # read line by line
        line = receipts.readline()
        data = json.loads(line)
    
        temp = pd.json_normalize(data)
        df_receipts = df_receipts.append(temp)
    
    except:        # break if last line
        break 

In [77]:
# rename columns
df_receipts.rename(columns={
                          '_id.$oid': '_id',
                          'createDate.$date':'createDate',
                          'dateScanned.$date': 'dateScanned',
                          'finishedDate.$date': 'finishedDate',
                          'modifyDate.$date': 'modifyDate',
                          'pointsAwardedDate.$date': 'pointsAwardedDate',
                          'purchaseDate.$date': 'purchaseDate'
                          }, 
                 inplace=True)

# reorder columns
column_names = ['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
                'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
                'pointsAwardedDate', 'pointsEarned', 'purchaseDate', 
                'purchasedItemCount', 'rewardsReceiptItemList', 
                'rewardsReceiptStatus', 'totalSpent', 'userId']

receipts = df_receipts.reindex(columns=column_names)

# converting unix to timestamp
receipts['createDate'] = pd.to_datetime(receipts['createDate'],unit='ms')
receipts['dateScanned'] = pd.to_datetime(receipts['dateScanned'],unit='ms')
receipts['finishedDate'] = pd.to_datetime(receipts['finishedDate'],unit='ms')
receipts['modifyDate'] = pd.to_datetime(receipts['modifyDate'],unit='ms')
receipts['pointsAwardedDate'] = pd.to_datetime(receipts['pointsAwardedDate'],unit='ms')
receipts['purchaseDate'] = pd.to_datetime(receipts['purchaseDate'],unit='ms')

# make a copy of original data
receipts_original = receipts.copy()

# drop rewardsReceiptItemList
receipts.drop('rewardsReceiptItemList', axis=1, inplace=True)

receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
0,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
0,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,5,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
0,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
0,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11.000,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaT,2021-03-01 10:22:28.000,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
0,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
0,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaN,NaT,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
0,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [82]:
receipts.to_csv('receipts.csv', index=False)

In [73]:
# null check
receipts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1119 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    object        
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptStatus     1119 non-null   object        
 12  totalSpent               684 non-null

In [71]:
# duplicate check
receipts.duplicated().sum()

0

## receipt_item

In [19]:
# isolate rewardsReceiptItemList
receipts_item = receipts_original[['_id', 'rewardsReceiptItemList']]

# change table from receipt-based to item-based
receipts_item = receipts_item.explode('rewardsReceiptItemList')
receipts_item = receipts_item.reset_index(drop=True)
receipts_item

,_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
2,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
3,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
4,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."
...,...,...
7376,603d0b710a720fde1000042a,NaN
7377,603cf5290a720fde10000413,NaN
7378,603ce7100a7217c72c000405,"{'barcode': 'B076FJ92M4', 'description': 'muel..."
7379,603ce7100a7217c72c000405,"{'barcode': 'B07BRRLSVC', 'description': 'thin..."


In [20]:
# flatten rewardsReceiptItemList
receipts_item = pd.concat(
    [receipts_item, json_normalize(receipts_item['rewardsReceiptItemList'])], axis=1)

receipts_item.drop('rewardsReceiptItemList', axis=1,  inplace=True)
receipts_item

,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,603cf5290a720fde10000413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,603ce7100a7217c72c000405,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [83]:
receipts_item.to_csv('receipts_item.csv', index=False)

In [37]:
# null check
receipts_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   _id                                 7381 non-null   object 
 1   barcode                             3090 non-null   object 
 2   description                         6560 non-null   object 
 3   finalPrice                          6767 non-null   object 
 4   itemPrice                           6767 non-null   object 
 5   needsFetchReview                    813 non-null    object 
 6   partnerItemId                       6941 non-null   object 
 7   preventTargetGapPoints              358 non-null    object 
 8   quantityPurchased                   6767 non-null   float64
 9   userFlaggedBarcode                  337 non-null    object 
 10  userFlaggedNewItem                  323 non-null    object 
 11  userFlaggedPrice                    299 non

In [72]:
# duplicate check
receipts_item.duplicated().sum()

0

## users

In [21]:
users = open("users.json", "r")

df_users = pd.DataFrame()

while True:
    try:           # read line by line
        line = users.readline()
        data = json.loads(line)
    
        temp = pd.json_normalize(data)
        df_users = df_users.append(temp)
    
    except:        # break if last line
        break

In [22]:
# rename columns
df_users.rename(columns={
                          '_id.$oid': '_id',
                          'createdDate.$date':'createdDate',
                          'lastLogin.$date': 'lastLogin',
                          }, 
                 inplace=True)

# reorder columns
column_names = ['_id', 'state', 'createdDate',
                'lastLogin', 'role', 'active', 'signUpSource']

users = df_users.reindex(columns=column_names)

# converting unix to timestamp
users['createdDate'] = pd.to_datetime(users['createdDate'],unit='ms')
users['lastLogin'] = pd.to_datetime(users['lastLogin'],unit='ms')

users

,_id,state,createdDate,lastLogin,role,active,signUpSource
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
0,5ff1e1eacfcf6c399c274ae6,WI,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,True,Email
0,5ff1e194b6a9d73a3a9f1052,WI,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,True,Email
...,...,...,...,...,...,...,...
0,54943462e4b07e684157a532,NaN,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000,fetch-staff,True,NaN
0,54943462e4b07e684157a532,NaN,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000,fetch-staff,True,NaN
0,54943462e4b07e684157a532,NaN,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000,fetch-staff,True,NaN
0,54943462e4b07e684157a532,NaN,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204000000,fetch-staff,True,NaN


In [84]:
users.to_csv('users.csv', index=False)

In [40]:
# null check
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 0
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           495 non-null    object        
 1   state         439 non-null    object        
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   active        495 non-null    bool          
 6   signUpSource  447 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 27.6+ KB


In [69]:
# duplicate check
users.duplicated().sum()

283

# Query

## When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [53]:
q = """
    SELECT rewardsReceiptStatus, COUNT(*)
    FROM receipts r
    GROUP BY rewardsReceiptStatus
    
"""

q1 = sqldf(q)
q1

,rewardsReceiptStatus,COUNT(*)
0,FINISHED,518
1,FLAGGED,46
2,PENDING,50
3,REJECTED,71
4,SUBMITTED,434


In [63]:
q = """
    SELECT rewardsReceiptStatus, AVG(totalSpent) average_spend
    FROM receipts r
    WHERE rewardsReceiptStatus == 'FINISHED' OR rewardsReceiptStatus == 'REJECTED'
    GROUP BY rewardsReceiptStatus
    ORDER BY AVG(totalSpent) DESC
"""

answer = sqldf(q)
answer

,rewardsReceiptStatus,average_spend
0,FINISHED,80.854305
1,REJECTED,23.326056


# Data Quality Issues

As explored above, these are some data quality issues:
- Many nested json, especially rewardsReceiptItemList column in Receipts table. Needs a lot of preprocessing and cleaning to flatten the whole structure.

- Many null values. Needs a lot of preprocessing.

- All date related columns are in UNIX format, need to be converted into datatime objects.

- There are duplicates in Users table.

# Message to stakeholders


Hello,

Hope all is well. As a data analyst, I was performing exploratory analysis on our receipts, users, and brands data. However, I discovered some data quality issues that need to be resolved before proceeding further on the analysis. 

1. There are duplicates in Users data. Please address this with data collection team in order to avoid creating duplicate users in our database.
2. There is a lot of missing data. For example, some items on our receipt data are not associated with any brand or barcode, which is concerning since these are important data piece for us to identify top brands. Capturing more can help us generate more insights and analysis.
3. Date related fields are in UNIX format, which is not a human readable format. A easy-to-read date format should be adopted in order to minimize processing error.
4. There are a lot of nested data, which requires longer processing time. Data should be stored in a flatter format for easier processing. 

I would love to connect and discuss these issues further. Please let me know a time to continue this conversation. 

Thank you for your time and I look forward to hearing back.

Warm regards,

Bowen